# Label The Candidates!

This notebook corresponds to labeling and genearting features for each extracted candidate from the [previous notebook](1.data-loader.ipynb).

## MUST RUN AT THE START OF EVERYTHING

Load all the imports and set up the database for database operations. Plus, set up the particular candidate type this notebook is going to work with. 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from collections import defaultdict
import csv
import os
import re


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm

In [2]:
#Set up the environment
username = "danich1"
password = "snorkel"
dbname = "pubmeddb"

#Path subject to change for different os
database_str = "postgresql+psycopg2://{}:{}@/{}?host=/var/run/postgresql".format(username, password, dbname)
os.environ['SNORKELDB'] = database_str

from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
from snorkel.annotations import FeatureAnnotator, LabelAnnotator
from snorkel.features import get_span_feats
from snorkel.models import candidate_subclass
from snorkel.models import Candidate, GoldLabel
from snorkel.viewer import SentenceNgramViewer

In [4]:
edge_type = "cg"
debug = False

In [5]:
if edge_type == "dg":
    DiseaseGene = candidate_subclass('DiseaseGene', ['Disease', 'Gene'])
    edge = "disease_gene"
elif edge_type == "gg":
    GeneGene = candidate_subclass('GeneGene', ['Gene1', 'Gene2'])
    edge = "gene_gene"
elif edge_type == "cg":
    CompoundGene = candidate_subclass('CompoundGene', ['Compound', 'Gene'])
    edge = "compound_gene"
elif edge_type == "cd":
    CompoundDisease = candidate_subclass('CompoundDisease', ['Compound', 'Disease'])
    edge = "compound_disease"
else:
    print("Please pick a valid edge type")

# Develop Label Functions

## Look at potential Candidates

Use this to look at loaded candidates from a given set. The constants represent the index to retrieve the appropiate set. Ideally, here is where one can look at a subset of the candidate and develop label functions for candidate labeling.

In [12]:
train_candidate_df = pd.read_excel("data/compound_gene/sentence-labels.xlsx")
train_candidate_df.head(2)

,candidate_id,compound,gene,drugbank_id,entrez_gene_id,sentence,label,name,gene_symbol,LF_HETNET_DRUGBANK,...,LF_RISK,LF_PATIENT_WITH,LF_PURPOSE,LF_CONCLUSION_TITLE,LF_NO_CONCLUSION,LF_CONCLUSION,LF_DG_DISTANCE_SHORT,LF_DG_DISTANCE_LONG,LF_DG_ALLOWED_DISTANCE,LF_NO_VERB
0,50628,proline,RNase A,DB02853,6035,At 0 degrees C the tyrosine-detected folding r...,0.037605,D-Proline,RNASE1,0,...,0,0,0,0,-1,0,0,0,1,0
1,50632,ornithine,catalase,DB00129,847,"The levels of superoxide anion production, orn...",0.298675,L-Ornithine,CAT,0,...,0,0,0,0,0,1,-1,0,0,0


In [13]:
train_candidate_ids = list(map(int, train_candidate_df.candidate_id.values))[0:10]

In [14]:
candidates = session.query(CompoundGene).filter(CompoundGene.id.in_(train_candidate_ids)).limit(100)
sv = SentenceNgramViewer(candidates, session)

<IPython.core.display.Javascript object>

In [15]:
sv

SentenceNgramViewer(cids=[[[9], [3], [0]], [[7], [4], [5]], [[2], [6], [8]], [[1]]], html='<head>\n<style>\nspan.candidate {\n    background-color: rgba(255,255,0,0.3);\n}\n\nspan.candidate-h {\n    background-color: rgba(255,255,0,1.0);\n}\n\nspan.true-candidate {\n    background-color: rgba(92,184,92,0.3);\n}\n\nspan.true-candidate-h {\n    background-color: rgba(92,184,92,1.0);\n}\n\nspan.false-candidate {\n    background-color: rgba(169,68,66,0.3);\n}\n\nspan.false-candidate-h {\n    background-color: rgba(169,68,66,1.0);\n}\n\nspan.highlighted {\n    font-weight: bolder;\n    box-shadow: 1px 1px 1px grey;\n}\n\nspan.left-candidate {\n    color: red;\n}\n\nspan.right-candidate {\n    color: blue;\n}\n\nspan.gold-annotation {\n    color: red;\n}\n\nspan.highlighted-gold-annotation {\n    background-color: rgba(217,83,79,0.5);\n}\n\n.panel-heading {\n    position: relative;\n}\n\n.panel-footer {\n    position: relative;\n}\n\n.panel-left-justify {\n    position: absolute;\n    right: 15px;\n    top: 20px;\n}\n\n.viewer-page {\n    display: none;\n}\n\n.help-text {\n    color: gray;\n    font-weight: 300;\n}\n</style>\n</head>\n\n<div id="viewer" class="viewer-pane panel panel-default" tabindex="0">\n    <div class="panel-heading">\n        <h3 class="panel-title"><i class="fa fa-eye fa-lg" aria-hidden="true"></i> Viewer</h3>\n        <div id="candidate-caption" class="panel-left-justify"></div>\n    </div>\n    <div class="panel-body" style="height:225px">\n<div class="viewer-page" id="viewer-page-0" style="display: block;">\n    <ul class="list-group">\n<li class="list-group-item" data-toggle="tooltip" data-placement="top" title="61680"><span class="">In order to examine the mechanism of the so-called "paradoxical decrease" in plasma GH with CB-154, GH responses to CB-154 were compared with GH responses to thyrotropin-releasing </span><span class="candidate 9 9-1">hormone</span><span class=""> (TRH), </span><span class="candidate 9 9-0">arginine</span><span class="">, and luteinizing hormone-releasing hormone (LH-RH) in 20 cases of acromegaly.</span></li>\n\n<li class="list-group-item" data-toggle="tooltip" data-placement="top" title="106716"><span class="">Serum prolactin response to </span><span class="candidate 3 3-0">thyrotropin-releasing</span><span class=""> </span><span class="candidate 3 3-1">hormone</span><span class=""> and metoclopramide in patients with prolactin-secreting tumors before and after transsphenoidal surgery.</span></li>\n\n<li class="list-group-item" data-toggle="tooltip" data-placement="top" title="142978"><span class="">The </span><span class="candidate 0 0-0">carnitine</span><span class=""> concentration correlated positively and statistically significantly with the activities of 3-OH-acyl-CoA dehydrogenase and citrate synthase, with the incorporation rate of palmitate-carbon into </span><span class="candidate 0 0-1">CO2</span><span class="">, and the incorporation rate of glucose-carbon into lactate in the muscle tissue.</span></li>\n</ul>\n</div>\n\n<div class="viewer-page" id="viewer-page-1">\n    <ul class="list-group">\n<li class="list-group-item" data-toggle="tooltip" data-placement="top" title="187180"><span class="">The (Mg2+ + Ca2+)</span><span class="candidate 7 7-1">-ATPase</span><span class=""> activity of the three preparations was stimulated to different extents by a </span><span class="candidate 7 7-0">Ca2+</span><span class="">-dependent protein activator isolated from hemolysates.</span></li>\n\n<li class="list-group-item" data-toggle="tooltip" data-placement="top" title="243372"><span class="">The levels of superoxide anion production, </span><span class="candidate 4 4-0">ornithine</span><span class=""> decarboxylase, </span><span class="candidate 4 4-1">catalase</span><span class=""> and ribonucleic acid in the tumor tissue are very different from those in the kidney.</span></li>\n\n<li class="list-group-item" data-toggle="tooltip" data-placement="top" title="310896"><span clas

In [16]:
c = sv.get_selected()
c

CompoundGene(Span("b'Ca2+'", sentence=187180, chars=[101,104], words=[22,23]), Span("b'-ATPase'", sentence=187180, chars=[17,23], words=[8,9]))

# Label Functions

Here is one of the fundamental part of this project. Below are the label functions that are used to give a candidate a label of 1,0 or -1 which corresponds to correct label, unknown label and incorrection label. The goal here is to develop functions that can label accurately label as many candidates as possible. This idea comes from the [data programming paradigm](https://papers.nips.cc/paper/6523-data-programming-creating-large-training-sets-quickly), where the goal is to be able to create labels that machine learning algorithms can use for accurate classification.  

In [6]:
if edge_type == "dg":
    from utils.disease_gene_lf import LFS, LF_DEBUG
elif edge_type == "gg":
    from utils.gene_gene_lf import *
elif edge_type == "cg":
    from utils.compound_gene_lf import LFS, LF_DEBUG, knowledge_base
elif edge_type == "cd":
    from utils.compound_disease_lf import *
else:
    print("Please pick a valid edge type")

# Label The Candidates

Label each candidate based on the provided labels above. This code runs with realtive ease, but optimization is definitely needed when the number of label functions increases linearly.

In [7]:
from  sqlalchemy.sql.expression import func
labeler = LabelAnnotator(lfs=list(LFS.values()))

### Train Set

In [ ]:
sql = '''
SELECT id from candidate
WHERE split = 6 and type='compound_gene'
ORDER BY RANDOM()
LIMIT 50000;
'''
target_cids = [x[0] for x in session.execute(sql)]

In [ ]:
target_cids

In [ ]:
np.savetxt('data/compound_gene/labeled_candidates.txt', target_cids)

### Dev Set

In [ ]:
sql = '''
SELECT candidate_id FROM gold_label
'''
gold_cids = [x[0] for x in session.execute(sql)]
gold_cids

In [ ]:
sql = '''
SELECT id from candidate
WHERE split = 6 and type='compound_gene'
ORDER BY RANDOM()
LIMIT 10000;
'''
gold_cids = [x[0] for x in session.execute(sql)]
gold_cids

In [ ]:
np.savetxt('data/compound_gene/labeled_dev_candidates.txt', gold_cids)

# Quickly Relabel Candidates

Use this block here to re-label candidates that have already been labled from the above process.

In [8]:
target_cids = np.loadtxt('data/compound_gene/labeled_candidates.txt').astype(int).tolist()

In [9]:
cids = session.query(CompoundGene.id).filter(CompoundGene.id.in_(target_cids))
%time L_train = labeler.apply(split=6, cids_query=cids, parallelism=5)

Clearing existing...
Running UDF...
CPU times: user 2min 49s, sys: 9.12 s, total: 2min 58s
Wall time: 8min 51s


In [10]:
#dev_df = pd.read_excel("data/sentence-labels-dev-hand-labeled.xlsx")
#dev_df = dev_df[dev_df.curated_dsh.notnull()]
#gold_cids = list(map(int, dev_df.candidate_id.values))
gold_cids = np.loadtxt('data/compound_gene/labeled_dev_candidates.txt').astype(int).tolist()
len(gold_cids)

10000

In [11]:
cids = session.query(Candidate.id).filter(Candidate.id.in_(gold_cids))
%time L_dev = labeler.apply_existing(cids_query=cids, parallelism=5, clear=False)

Running UDF...
CPU times: user 3min 18s, sys: 5.54 s, total: 3min 23s
Wall time: 5min 4s


In [ ]:
sql = '''
SELECT candidate_id FROM gold_label
INNER JOIN Candidate ON Candidate.id=gold_label.candidate_id
WHERE Candidate.split=0;
'''
cids = session.query(Candidate.id).filter(Candidate.id.in_([x[0] for x in session.execute(sql)]))
%time L_train_hand_labeled = labeler.apply_existing(cids_query=cids, parallelism=5, clear=False)